In [2]:
import os
import glob
import torch
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from oml.models import ViTExtractor
from oml.registry import get_transforms_for_pretrained
import torch.nn.functional as F

import yaml
from oml.registry.models import get_extractor_by_cfg

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
transform, _ = get_transforms_for_pretrained("vitb14_dinov2")
model_path = "./weights/birds_200_vitb14_dinov2/checkpoints/best.ckpt"

extractor = get_extractor_by_cfg({
    'name': 'vit',
    'args': {
        'normalise_features': True,
        'use_multi_scale': False,
        'weights': model_path,
        'arch': 'vitb14'
    }
}).to(DEVICE)

def get_embedding(image_path):
    with torch.no_grad():
        image = Image.open(image_path)
        image = transform(image).unsqueeze(0)
        embeddings = extractor(image.to(DEVICE))
        return embeddings

embedding1 = get_embedding("./downloads/birds-200-species/CUB_200_2011/train/001.Black_footed_Albatross/Black_Footed_Albatross_0001_796111.jpg")
embedding2 = get_embedding("./downloads/birds-200-species/CUB_200_2011/train/001.Black_footed_Albatross/Black_Footed_Albatross_0058_796074.jpg")
embedding3 = get_embedding("./downloads/birds-200-species/CUB_200_2011/train/078.Gray_Kingbird/Gray_Kingbird_0051_70139.jpg")

F.cosine_similarity(embedding1, embedding2, dim=1).item(), F.cosine_similarity(embedding1, embedding3, dim=1).item()

Prefix <model.model.> was removed from the state dict.


(0.803785502910614, 0.060847196727991104)